# 使用机器学习的书本推荐系统

In [8]:
import pandas as pd

# 加载数据集
data_path = 'pivot_use.csv'
data = pd.read_csv(data_path)

# 显示数据的前几行以了解其结构
data.head()


,user_id,title_without_series,bert_rating
0,cfa37f8ff40f4c05fe58462a0344aa27,"Scarlet (The Lunar Chronicles, #2)",5
1,6a3a8c4abf9dc036d2a652fd364a4556,"New Moon (Twilight, #2)",4
2,3d7107325f0bd2bd27a34904aededac3,"Winter (The Lunar Chronicles, #4)",3
3,e7a00ca7bc7dc46217540ffce134f573,"Angelfall (Penryn & the End of Days, #1)",5
4,f05cc5cae2060dbb24fa88313b28783f,Let the Right One In,4


数据预处理：
将user_id和title_without_series转换为整数索引。
分割数据集为训练集和测试集。

模型架构：
用户嵌入层：学习用户的嵌入表示。
书籍嵌入层：学习书籍的嵌入表示。
合并层：将用户和书籍的嵌入表示合并。
全连接层：对合并后的嵌入进行处理，预测评分。

训练模型：
使用均方误差（MSE）作为损失函数。
选择适合的优化器，例如Adam。

评估和推荐：
评估模型在测试集上的性能。
对于给定的user_id，模型预测该用户对所有书籍的评分，然后选择评分最高的几本书作为推荐。

In [9]:
# 数据预处理
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 初始化标签编码器
user_le = LabelEncoder()
item_le = LabelEncoder()

# 对user_id和title_without_series进行编码
data['user_id_enc'] = user_le.fit_transform(data['user_id'])
data['title_enc'] = item_le.fit_transform(data['title_without_series'])

# 分割数据集为训练集和测试集
train, test = train_test_split(data, test_size=0.2, random_state=42)

# 查看处理后的数据集和分割结果
train.head(), 
train.shape, 
test.shape


(228308, 5)

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from tensorflow.keras.optimizers import Adam

# 确定嵌入向量的大小
embedding_size = 8

# 用户嵌入模型
user_input = Input(shape=(1,), name='user_input')
# 注意这里使用了总的唯一用户数174980
user_embedding = Embedding(input_dim=174980, output_dim=embedding_size, name='user_embedding')(user_input)
user_vec = Flatten(name='user_flatten')(user_embedding)

# 书籍嵌入模型
item_input = Input(shape=(1,), name='item_input')
# 注意这里使用了总的唯一书籍数119868
item_embedding = Embedding(input_dim=119868, output_dim=embedding_size, name='item_embedding')(item_input)
item_vec = Flatten(name='item_flatten')(item_embedding)

# 合并两个嵌入向量
merged = Concatenate()([user_vec, item_vec])

# 全连接层
dense = Dense(128, activation='relu')(merged)
dense = Dense(64, activation='relu')(dense)
output = Dense(1)(dense)

# 定义和编译模型
model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

# 查看模型概览
model.summary()



Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 8)         1399840     user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 8)         958944      item_input[0][0]                 
____________________________________________________________________________________________

In [13]:
# 为所有书籍生成给定用户的评分预测
import pandas

def make_prediction(user_id):
    user_idx = user_le.transform([user_id]) # 转换用户ID
    item_idxs = np.array(range(data['title_enc'].nunique())) # 所有书籍的索引
    user_idxs = np.array([user_idx] * len(item_idxs))
    predictions = model.predict([user_idxs, item_idxs])
    top_items = predictions.flatten().argsort()[-10:][::-1] # 获取最高评分的10本书
    return item_le.inverse_transform(top_items) # 转换回书籍标题

# 示例：为特定用户生成推荐
user_id = '6a3a8c4abf9dc036d2a652fd364a4556'
recommended_titles = make_prediction(user_id)
print(recommended_titles)


['The Knight of Disks (Villainess Book 4)' 'Where The Dead Ones Play'
 'Albino' 'Fanning the Flames (Going Down in Flames, #4)'
 'Everblue (Mer Tales #1)' 'The First Moon (The Pack Claims a Mate #2)'
 'Secret Of The Rose (Legacy Of Magick, #2)' 'Drácula, el no muerto'
 'Tsubasa RESERVoir CHRoNiCLE, Volume 21 (Tsubasa Reservoir Chronicle, #21)'
 'The House on Blackstone Moor (The Blackstone Vampires, #1)']


(174980, 119868)